In [24]:
import json
import os
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-RS'

In [3]:
os.chdir('./../airbnb-data')

In [5]:
with open('review_raw.json','r',encoding='utf-8') as fp:
    data = json.load(fp)

In [6]:
print('리뷰가 달린 숙소의 수: {}'.format(len(review))) # 데이터 수집중

리뷰가 달린 숙소의 수: 28837


기본적인 추천 시스템 구현을 위해

__room, user, rating__ 을 컬럼으로하는 테이블을 짜보자

In [21]:
list(data.items())[0]

('10030136',
 [{'collection_tag': None,
   'comments': '친구들이랑 6명이 갔는데\n숙소 위치도 너무 좋고 방도 깨끗해서\n다들 맘에 들어했어요!!\n에어비엔비를  처음 이용해봤는데\n너무 좋아서 다음번에도 이용할것같아요\n해운대해변이랑도 걸어갈수 있고\n연락도 너무 잘됐어요',
   'created_at': '2016-10-08T23:57:42Z',
   'id': 106917581,
   'language': 'ko',
   'localized_date': '2016년 10월',
   'rating': 5,
   'response': '',
   'reviewee': {'deleted': False,
    'first_name': 'Jaewook',
    'host_name': 'Jaewook',
    'id': 38530724,
    'is_superhost': False,
    'picture_url': 'https://a0.muscache.com/im/users/38530724/profile_pic/1436955527/original.jpg?aki_policy=profile_x_medium',
    'profile_path': '/users/show/38530724'},
   'reviewer': {'deleted': False,
    'first_name': 'Joohee',
    'host_name': 'Joohee',
    'id': 96535213,
    'is_superhost': False,
    'picture_url': 'https://a0.muscache.com/im/pictures/32cbca58-85f2-47e5-b09c-a5ead7662304.jpg?aki_policy=profile_x_medium',
    'profile_path': '/users/show/96535213'}},
  {'collection_tag': None,
   'comments': '房间很大很

In [27]:
data_tmp = []
idx = 0
for room_id,rev_lst in data.items():
    for rev in rev_lst:
        user_id = rev['reviewer']['id']
        rating = rev['rating']
        tpl_tmp = (idx,room_id,user_id,rating)
        data_tmp.append(tpl_tmp)
        idx += 1

In [30]:
df = pd.DataFrame(data=data_tmp,columns=['idx','room_id','user_id','rating'])
df.head()

,idx,room_id,user_id,rating
0,0,10030136,96535213,5
1,1,10030136,92096013,5
2,2,10030136,91797253,5
3,3,10030136,37297318,5
4,4,10030136,55334719,4


In [37]:
df.user_id = df.user_id.astype('object')
df.rating = df.rating.astype('int8')
df.idx = df.idx.astype('object')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581403 entries, 0 to 581402
Data columns (total 4 columns):
idx        581403 non-null object
room_id    581403 non-null object
user_id    581403 non-null object
rating     581403 non-null int8
dtypes: int8(1), object(3)
memory usage: 13.9+ MB


In [39]:
df.describe()

,rating
count,581403.000000
mean,4.694468
std,0.703014
min,0.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


In [40]:
df.room_id.value_counts().iloc[:10] # rating한 유저 수가 가장 많은 room top10

6261834     398
5364656     344
6037162     342
6825974     342
7793048     325
6825848     293
10367215    292
3651105     290
5127584     290
5164709     288
Name: room_id, dtype: int64

In [41]:
df.groupby('room_id').mean()[['rating']].sort_values('rating',ascending=False).iloc[:15,:] # 평점평균의  top 15

,rating
room_id,
28978936,5.0
25127434,5.0
25113837,5.0
7861234,5.0
25114754,5.0
25114802,5.0
17030662,5.0
25114996,5.0
25115219,5.0


In [42]:
df.groupby('user_id').size().sort_values(ascending=False).iloc[:10] # 유저별 rating 수 top10

user_id
59846539     53
19937135     33
32314343     32
146595339    31
2975732      31
54604405     29
8461756      28
27920405     27
83589526     27
109963105    27
dtype: int64

In [45]:
airbnb_stats = df.groupby('room_id').agg({'rating':[np.size,np.mean]}) #size와 mean 한번에 
airbnb_stats.head(10)

rating          
           size      mean
room_id                  
10003340     12  4.750000
10008511      4  4.000000
10010172     20  4.750000
10010532    134  4.746269
1001751      31  4.935484
1001777      24  5.000000
1001811      76  4.736842
10022410     52  4.653846
10024115     60  4.916667
10024477     59  4.813559

In [46]:
atleast_50 = airbnb_stats['rating']['size'] >=50 # size 50 이상이라는 masking줘서 선별
airbnb_stats[atleast_50].sort_values(by=[('rating','mean')], ascending= False)

rating          
           size      mean
room_id                  
21466198     51  5.000000
15529380     56  5.000000
18441135     58  5.000000
27265592     51  5.000000
13668072     64  5.000000
14996953     80  5.000000
20058294     79  5.000000
21552767     52  5.000000
16520824     62  5.000000
8653827      52  5.000000
12982266     78  5.000000
4258317     139  4.992806
16458614    103  4.990291
6516510     100  4.990000
8409888      98  4.989796
18674482     93  4.989247
16464138     92  4.989130
13064792     86  4.988372
18163533     74  4.986486
21328230     73  4.986301
17179784     71  4.985915
19837447     69  4.985507
15892773     65  4.984615
2756508      64  4.984375
2548014      64  4.984375
7667024      63  4.984127
18776802     63  4.984127
17758997     63  4.984127
7666781      63  4.984127
22413341    126  4.984127
...         ...       ...
13885242     53  4.075472
8177951      54  4.074074
5441934      68  4.073529
14168381     69  4.072464
13644366    106  4.066038
15971627     77  4.064935
305641       90  4.055556
3975836      95  4.052632
23054079     61  4.049180
16993176     51  4.039216
17463995     53  4.037736
18583018     58  4.017241
15635299     60  4.016667
1867920      60  4.000000
822899       55  4.000000
2428360      91  3.989011
11618416     50  3.960000
4857899      58  3.948276
3824018      54  3.944444
6026644      50  3.940000
17148000     63  3.920635
7887825      61  3.918033
9614458      70  3.914286
4348169     133  3.902256
2521780      61  3.885246
728951       52  3.884615
1061585      82  3.731707
17855531     57  3.701754
385906       80  3.512500
7330772      66  3.000000

[3853 rows x 2 columns]

In [ ]:
# 우선 cross tab이용해서 wide하게 만들어보자
# IndexError: index 1654307388 is out of bounds for axis 0 with size 1654287474


In [47]:
room_crosstab = pd.pivot_table(data=df, values='rating', index='user_id',columns='room_id')
room_crosstab.head()

IndexError: index 1654307388 is out of bounds for axis 0 with size 1654287474